# 1. 预测全家桶与机器学习四大神器

## Attrition

### Catboost

In [ ]:
import pandas as pd

In [15]:
train=pd.read_csv('~/data/BI/train.csv',index_col=0)
test=pd.read_csv('~/data/BI/test.csv',index_col=0)

In [16]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [17]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [18]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)

In [19]:
import catboost as cb
import numpy as np
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [21]:
model = cb.CatBoostClassifier(iterations=1000, 
                              depth=7, 
                              learning_rate=0.01, 
                              loss_function='Logloss', 
                              eval_metric='AUC',
                              logging_level='Verbose', 
                              metric_period=50
                             )

In [22]:
# 得到分类特征的列号
categorical_features_indices = []
for i in range(len(X_train.columns)):
    if X_train.columns.values[i] in attr:
        categorical_features_indices.append(i)
print(categorical_features_indices)

[0, 1, 3, 5, 6, 9, 13, 15, 19, 20]


In [23]:
model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=categorical_features_indices)

0:	test: 0.6390374	best: 0.6390374 (0)	total: 65.8ms	remaining: 1m 5s
50:	test: 0.7893703	best: 0.7893703 (50)	total: 216ms	remaining: 4.01s
100:	test: 0.7950453	best: 0.7950453 (100)	total: 386ms	remaining: 3.44s
150:	test: 0.8011568	best: 0.8011568 (150)	total: 560ms	remaining: 3.15s
200:	test: 0.7958092	best: 0.8011568 (150)	total: 752ms	remaining: 2.99s
250:	test: 0.8023573	best: 0.8023573 (250)	total: 947ms	remaining: 2.82s
300:	test: 0.8034487	best: 0.8034487 (300)	total: 1.12s	remaining: 2.6s
350:	test: 0.8046491	best: 0.8046491 (350)	total: 1.3s	remaining: 2.4s
400:	test: 0.8030121	best: 0.8046491 (350)	total: 1.53s	remaining: 2.29s
450:	test: 0.8054131	best: 0.8054131 (450)	total: 1.73s	remaining: 2.11s
500:	test: 0.8067227	best: 0.8067227 (500)	total: 1.93s	remaining: 1.92s
550:	test: 0.8062862	best: 0.8067227 (500)	total: 2.1s	remaining: 1.72s
600:	test: 0.8056313	best: 0.8067227 (500)	total: 2.29s	remaining: 1.52s
650:	test: 0.8053039	best: 0.8067227 (500)	total: 2.47s	rema

In [24]:
#model = cb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test)
#predict = model.predict_proba(test)
#print(predict)
test['Attrition']=predict

In [25]:
## 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
test[['Attrition']].to_csv('~/data/course_data/submit_cb.csv')

### GBDT

In [7]:
import pandas as pd

In [8]:
train=pd.read_csv('~/data/BI/train.csv',index_col=0)
test=pd.read_csv('~/data/BI/test.csv',index_col=0)

In [29]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [30]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [31]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#train.to_csv('temp.csv')
#print(train)

In [32]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [34]:
# 采用回归算法，可以得到更好的AUC结果
model = GradientBoostingRegressor(random_state=10)
model.fit(X_train, y_train)
predict = model.predict(test)
print(predict)
test['Attrition']=predict
#print(predict)

[ 1.01678366e-01  4.47445681e-02  1.42121556e-01  7.57204880e-02
  7.03200210e-01  3.70055651e-01  3.74376235e-01  4.37242139e-02
 -1.15634448e-01  2.97987772e-01  1.20779518e-01  1.09366950e-01
  1.07679394e-01  7.66600582e-01  7.51844882e-02 -4.76111832e-02
  4.28285053e-02 -1.09673242e-02  1.59486732e-01  9.62417811e-02
  6.47325348e-01  1.10040397e-01 -9.94554567e-03 -4.55439809e-02
  2.68815031e-01  3.37430996e-01  7.35105859e-02  1.06048931e-01
  5.50770434e-01  4.15464265e-02  1.79826162e-01  1.01207422e-03
  1.58722155e-01  1.58671176e-01  8.62998751e-02  2.67636183e-02
  1.24005756e-01  9.94003577e-02  8.73177129e-02  1.59141438e-03
  3.02853331e-02 -5.02317582e-03 -2.69498786e-03  8.75329962e-03
  3.69313030e-02  5.03418977e-01  2.62794122e-01 -1.16559089e-02
  8.28838190e-01  4.78927949e-01  2.30745577e-01  3.61855221e-01
  1.97193403e-01  2.08480315e-01  4.27257886e-01  7.31510619e-02
 -4.10414290e-02  1.41280282e-01 -5.16541193e-02  4.63709471e-01
  8.91047041e-03  1.57167

In [35]:
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
test[['Attrition']].to_csv('~/data/course_data/submit_gbdt.csv')
print('submit_gbdt.csv saved')


submit_gbdt.csv saved


### LGB

In [3]:
import pandas as pd

In [4]:
train=pd.read_csv('~/data/BI/train.csv',index_col=0)
test=pd.read_csv('~/data/BI/test.csv',index_col=0)

In [5]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [6]:

# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [7]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#train.to_csv('../../resource/course_data_BI/temp.csv')
#print(train)

In [8]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [9]:
# param = {
#     'num_leaves':41,
#     'boosting_type': 'gbdt',
#     'objective':'binary',
#     'max_depth':15,
#     'learning_rate':0.001,
#     'metric':'binary_logloss'}
param = {'boosting_type':'gbdt',
                        'objective' : 'binary', #
                        # 'metric' : 'binary_logloss',
                        'metric' : 'auc',
                        # 'metric' : 'self_metric',
                        'learning_rate' : 0.01,
                        'max_depth' : 15,
                        'feature_fraction':0.8,
                        'bagging_fraction': 0.9,
                        'bagging_freq': 8,
                        'lambda_l1': 0.6,
                        'lambda_l2': 0,
                        # 'scale_pos_weight':k,
                        # 'is_unbalance':True
        }
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)


model = lgb.train(param,train_data,valid_sets=[train_data,valid_data],num_boost_round = 10000 ,early_stopping_rounds=200,verbose_eval=25, categorical_feature=attr)
predict=model.predict(test)
#print(predict)
test['Attrition']=predict
test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test['Attrition']=predict
test[['Attrition']].to_csv('~/data/course_data/submit_lgb.csv')


/Users/lilithgames/miniforge3/envs/tf/lib/python3.9/site-packages/lightgbm/basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Age', 'BusinessTravel', 'Department', 'Education', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 139, number of negative: 801
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002414 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1138
[LightGBM] [Info] Number of data points in the train set: 940, number of used features: 30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.147872 -> initscore=-1.751387
[LightGBM] [Info] Start training from score -1.751387
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't impro

/Users/lilithgames/miniforge3/envs/tf/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/lilithgames/miniforge3/envs/tf/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Exception ignored on calling ctypes callback function: <function _log_callback at 0x2826b3280>
Traceback (most recent call last):
  File "/Users/lilithgames/miniforge3/envs/tf/lib/python3.9/site-packages/lightgbm/basic.py", line 76, in _log_callback
    def _log_callback(msg):
KeyboardInterrupt: 


No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

In [ ]:
## Test xgboost

In [2]:
from xgboost import XGBClassifier
import pandas as pd
import warnings
%pylab inline

warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [3]:
# load data from url
df = pd.read_csv('~/data/Titanic.txt', sep=',', quotechar='"', encoding='ISO 8859-15')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   row.names  1313 non-null   int64  
 1   pclass     1313 non-null   object 
 2   survived   1313 non-null   int64  
 3   name       1313 non-null   object 
 4   age        633 non-null    float64
 5   embarked   821 non-null    object 
 6   home.dest  754 non-null    object 
 7   room       77 non-null     object 
 8   ticket     69 non-null     object 
 9   boat       347 non-null    object 
 10  sex        1313 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 113.0+ KB


,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male


In [4]:
# Filter some features
features = df[['pclass', 'age', 'sex']]
# Label
label = df['survived']
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  1313 non-null   object 
 1   age     633 non-null    float64
 2   sex     1313 non-null   object 
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


In [5]:
# Missing values ​​are filled with mean
features['age'].fillna(df['age'].mean(), inplace=True)
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  1313 non-null   object 
 1   age     1313 non-null   float64
 2   sex     1313 non-null   object 
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


In [6]:
# Divide the dataset
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(features, label, test_size = 0.25, random_state=33)

In [7]:
# Feature vectorization
from sklearn.feature_extraction import DictVectorizer

vec = DictVectorizer(sparse = False)
train_x = vec.fit_transform(train_x.to_dict(orient='record'))
test_x = vec.transform(test_x.to_dict(orient='record'))

In [8]:
# Random forest training and prediction
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(train_x, train_y)
print('The accuracy of random Forest Classifier on testing set:', rfc.score(test_x, test_y))

"""
The accuracy of random Forest Classifier on testing set: 0.7781155015197568
"""

The accuracy of random Forest Classifier on testing set: 0.7811550151975684


'\nThe accuracy of random Forest Classifier on testing set: 0.7781155015197568\n'

In [10]:
# xgboost training and prediction
from xgboost import XGBClassifier
xb = XGBClassifier()
print(xb)
xb.fit(train_x, train_y)
# print(f'The accuracy:', xb.score(test_x, test_y))
# """
# The accuracy: 0.7750759878419453
# """

In [1]:
print(f'accuracy:', xb.score(test_x, test_y))

NameError: name 'xb' is not defined